### __Importing the Dependencies__

In [22]:
!pip install category_encoders

import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders as ce
from utils import encoding


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### __Loading the dataset__

In [23]:
use_cols = [
    'Pclass', 'Sex', 'Age', 'Fare', 'SibSp',
    'Survived'
]

data = pd.read_csv('../datasets/titanic.csv', usecols=use_cols)
data.head()

,Survived,Pclass,Sex,Age,SibSp,Fare
0,0,3,male,22.0,1,7.2500
1,1,1,female,38.0,1,71.2833
2,1,3,female,26.0,0,7.9250
3,1,1,female,35.0,1,53.1000
4,0,3,male,35.0,0,8.0500


In [24]:
X_train, X_test, y_train, y_test = train_test_split(data, data.Survived, test_size=0.3, random_state=0)

In [25]:
X_train.shape, X_test.shape

((623, 6), (268, 6))

### __One-hot encoding__

In [26]:
# Replace the categorical variable by different boolean variables (0/1) to indicate whether or not certain label is true for that observation
data1 = pd.get_dummies(data,drop_first=True)

In [27]:
data1.head()

,Survived,Pclass,Age,SibSp,Fare,Sex_male
0,0,3,22.0,1,7.2500,True
1,1,1,38.0,1,71.2833,False
2,1,3,26.0,0,7.9250,False
3,1,1,35.0,1,53.1000,False
4,0,3,35.0,0,8.0500,True


### __Ordinal encoding__

In [28]:
# Replacing the labels by some ordinal number if ordinal is meaningful
ord_enc = ce.OrdinalEncoder(cols=['Sex']).fit(X_train,y_train)

In [29]:
data4 = ord_enc.transform(data)
print(data4.head(5))

   Survived  Pclass  Sex   Age  SibSp     Fare
0         0       3    1  22.0      1   7.2500
1         1       1    2  38.0      1  71.2833
2         1       3    2  26.0      0   7.9250
3         1       1    2  35.0      1  53.1000
4         0       3    1  35.0      0   8.0500


### __Mean encoding__

In [30]:
# Replace the label by the mean of the target for that label. (the target must be 0/1 valued or continuous)
# Cross check-- the mean of target group by Sex
X_train['Survived'].groupby(data['Sex']).mean()

Sex
female    0.753488
male      0.196078
Name: Survived, dtype: float64

In [31]:
mean_enc = encoding.MeanEncoding(cols=['Sex']).fit(X_train,y_train)

In [32]:
data6 = mean_enc.transform(data)
print(data6.head(5))

   Survived  Pclass       Sex   Age  SibSp     Fare
0         0       3  0.196078  22.0      1   7.2500
1         1       1  0.753488  38.0      1  71.2833
2         1       3  0.753488  26.0      0   7.9250
3         1       1  0.753488  35.0      1  53.1000
4         0       3  0.196078  35.0      0   8.0500


### __Target encoding__

In [33]:
# Similar to mean encoding, but use both posterior probability and prior probability of the target
# Create the encoder and fit with our data
target_enc = ce.TargetEncoder(cols=['Sex']).fit(X_train,y_train)

In [34]:
# Perform transformation data.Survived.groupby(data['Sex']).agg(['mean'])
data2 = target_enc.transform(data)

In [35]:
# Check the result
data2.head()

,Survived,Pclass,Sex,Age,SibSp,Fare
0,0,3,0.196078,22.0,1,7.2500
1,1,1,0.753488,38.0,1,71.2833
2,1,3,0.753488,26.0,0,7.9250
3,1,1,0.753488,35.0,1,53.1000
4,0,3,0.196078,35.0,0,8.0500


### __WOE encoding__

In [36]:
# Replace the label  with Weight of Evidence of each label. WOE is computed from the basic odds ratio
woe_enc = ce.WOEEncoder(cols=['Sex']).fit(X_train,y_train)

In [37]:
data3 = woe_enc.transform(data)

In [38]:
data3.head(5)

,Survived,Pclass,Sex,Age,SibSp,Fare
0,0,3,-0.950742,22.0,1,7.2500
1,1,1,1.555633,38.0,1,71.2833
2,1,3,1.555633,26.0,0,7.9250
3,1,1,1.555633,35.0,1,53.1000
4,0,3,-0.950742,35.0,0,8.0500
